# Prophet - Predecir el valor de Bitcoin


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Bitcoin, una criptomoneda. Es la criptomoneda que le ha marcado el camino a todas las demás que llegaron después utilizando su tecnología.  

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>


In [16]:
#Dependencias de Prophet
!pip install pystan==2.19.1.1 --quiet
!pip install cmdstanpy --quiet
# Instalamos el modelo predictivo Prophet
!pip install fbprophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

In [17]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de BTC lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [18]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today = '2023-04-19'
start_date = '2016-01-01'
# Descargar el dataframe
dis_df = yf.download('DIS',start_date, today).reset_index()

dis_df.tail()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
1830,2023-04-12,$101.25,$102.22,$97.70,$97.94,$97.94,9289200
1831,2023-04-13,$98.51,$101.07,$98.51,$100.84,$100.84,8745000
1832,2023-04-14,$100.50,$100.86,$98.93,$99.90,$99.90,7333400
1833,2023-04-17,$99.57,$100.57,$99.15,$100.30,$100.30,6705300
1834,2023-04-18,$100.61,$101.15,$99.60,$100.93,$100.93,8163900


In [19]:
# Validamos que no hay datos vacíos

# Validar si la columna 'Date' no tiene datos vacíos
if dis_df['Date'].notna().all():
    print("La columna 'Date' no tiene datos vacíos.")
else:
    print("La columna 'Date' tiene datos vacíos.")

# Validar si la columna 'Open' no tiene datos vacíos
if dis_df['Open'].notna().all():
    print("La columna 'Open' no tiene datos vacíos.")
else:
    print("La columna 'Open' tiene datos vacíos.")    

La columna 'Date' no tiene datos vacíos.
La columna 'Open' no tiene datos vacíos.


In [21]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
# Date será la fecha del valor y Open el valor del BTC en ese momento
df = dis_df.loc[:, ["Date", "Open"]]

# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
# TAREA HECHA: Crear un diccionario con clave "Date" y valor "ds"
# Y otra clave "Open" y valor "y"
new_names = {
    "Date": "ds", 
    "Open": "y",
}

# TAREA: Utiliza el diccionario para renombrar las columnas de df
df = df.rename (columns = new_names)

In [22]:
#Ejecuta la siguiente línea, si lo has hecho bien hasta aquí no dará error
df['ds'] = df['ds'].dt.tz_localize(None)

In [ ]:
df.tail()

,ds,y
1798,2022-10-12 00:00:00+00:00,"$1,279.73"
1799,2022-10-13 00:00:00+00:00,"$1,294.92"
1800,2022-10-14 00:00:00+00:00,"$1,288.05"
1801,2022-10-15 00:00:00+00:00,"$1,297.31"
1802,2022-10-16 00:00:00+00:00,"$1,275.01"


In [23]:
# Crear la gráfica del precio de apertura
#Asigna a la variable x la columna ds
x = df["ds"]
#Asigna a la variable y la columna y
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título a la gráfica
titulo = "Serie temporal de Disney Precio de apertura"
fig.update_layout(
    title_text=titulo,
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Ahora que has visto en la gráfica de que datos partimos. Inicializamos el Prophet y lo entramos pasandole el dataframe
</p>



In [40]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative"
from fbprophet import Prophet
model = Prophet(seasonality_mode='multiplicative') 

In [41]:
#TAREA: Entrena el modelo m con el dataframe de datos df

model.fit(df)


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


<p>
    Le decimos al modelo m los días en futuro que queremos predecir, en este caso el próximo año.
</p>



In [37]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe()
future = model.make_future_dataframe(365)
future.tail()

,ds
2195,2024-04-13
2196,2024-04-14
2197,2024-04-15
2198,2024-04-16
2199,2024-04-17


In [42]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines
# de semana, descomenta la siguiente línea:

future = future[ future['ds'].dt.dayofweek < 5 ] # Nos elimina los fines de semana de 'ds' para no predecirlos
future

,ds
0,2016-01-04
1,2016-01-05
2,2016-01-06
3,2016-01-07
4,2016-01-08
...,...
2193,2024-04-11
2194,2024-04-12
2197,2024-04-15
2198,2024-04-16


<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [43]:
#Utiliza el modelo m para predecir el dataframe future
forecast = model.predict(future)

In [44]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2091,2024-04-11,$32.95,$-151.06,$195.81
2092,2024-04-12,$32.81,$-147.88,$200.80
2093,2024-04-15,$32.40,$-153.40,$199.50
2094,2024-04-16,$32.25,$-150.95,$201.35
2095,2024-04-17,$32.02,$-154.17,$200.78


<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predecidos.
</p>



In [46]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
forecast[forecast['ds'] == next_day]['yhat'].item()
plot_plotly(model, forecast)

In [47]:
plot_components_plotly(model, forecast)